## Dicionário de dados

* **`transaction_id`**
    * **Descrição:** Identificador numérico único para cada transação.
    * **Valores:** Números inteiros. Ex: `21320398`.

* **`merchant_id`**
    * **Descrição:** Identificador numérico único para cada lojista onde a compra foi realizada.
    * **Valores:** Números inteiros. Ex: `29744`.

* **`user_id`**
    * **Descrição:** Identificador numérico único para cada cliente cadastrado.
    * **Valores:** Números inteiros. Ex: `97051`.

* **`card_number`**
    * **Descrição:** Número do cartão de crédito utilizado na transação (parcialmente oculto).
    * **Valores:** Sequência de números mascarada. Ex: `434505******9116`.

* **`transaction_date`**
    * **Descrição:** Data e hora exatas em que a transação ocorreu.
    * **Valores:** Data e hora em formato ISO. Ex: `2019-12-01T23:16:32.812632`.

* **`transaction_amount`**
    * **Descrição:** O valor monetário da transação.
    * **Valores:** Números decimais (float). Ex: `374.56`.

* **`device_id`**
    * **Descrição:** Identificador numérico único do dispositivo (ex: celular, computador) usado para fazer a transação.
    * **Valores:** Números inteiros (Ex: `285475`) ou valores nulos/ausentes.

* **`has_cbk`**
    * **Descrição:** Variável que indica se a transação sofreu um estorno por fraude (`True`) ou se foi uma transação legítima (`False`).
    * **Valores:** `True` (transação fraudulenta) ou `False` (transação legítima).

## Análise Exploratória dos Dados

### 1. Leitura dos dados

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
url = 'https://gist.githubusercontent.com/cloudwalk-tests/76993838e65d7e0f988f40f1b1909c97/raw/295d9f7cb8fdf08f3cb3bdf1696ab245d5b5c1c9/transactional-sample.csv'

In [3]:
df = pd.read_csv(url)

In [4]:
df.head()

,transaction_id,merchant_id,user_id,card_number,transaction_date,transaction_amount,device_id,has_cbk
0,21320398,29744,97051,434505******9116,2019-12-01T23:16:32.812632,374.56,285475.0,False
1,21320399,92895,2708,444456******4210,2019-12-01T22:45:37.873639,734.87,497105.0,True
2,21320400,47759,14777,425850******7024,2019-12-01T22:22:43.021495,760.36,NaN,False
3,21320401,68657,69758,464296******3991,2019-12-01T21:59:19.797129,2556.13,NaN,True
4,21320402,54075,64367,650487******6116,2019-12-01T21:30:53.347051,55.36,860232.0,False


Inicialmente, podemos notar que:
*   Temos 4 colunas de identificação (`transaction_id`, `merchant_id`, `user_id` e `device_id`);
*   A coluna `device_id` possui valores nulos;
*   Temos a data e hora da compra na coluna `transaction_date`;
*   Temos o valor da compra na coluna `transaction_amount`;
*   A coluna `has_cbk` é a coluna que define se a transação é fradulenta ou não.




### 2. Exploração dos dados

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3199 entries, 0 to 3198
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   transaction_id      3199 non-null   int64  
 1   merchant_id         3199 non-null   int64  
 2   user_id             3199 non-null   int64  
 3   card_number         3199 non-null   object 
 4   transaction_date    3199 non-null   object 
 5   transaction_amount  3199 non-null   float64
 6   device_id           2369 non-null   float64
 7   has_cbk             3199 non-null   bool   
dtypes: bool(1), float64(2), int64(3), object(2)
memory usage: 178.2+ KB


In [6]:
# Porcentagem dos valores nulos
(df.isnull().sum() / df.shape[0] * 100).round(2)

,0
transaction_id,0.00
merchant_id,0.00
user_id,0.00
card_number,0.00
transaction_date,0.00
transaction_amount,0.00
device_id,25.95
has_cbk,0.00


A ausência de valores na coluna `device_id` pode significar duas coisas: 1. problema na coleta de dados; 2. tentativa de esconder o dispositivo propositalmente.

Nesse caso, é importante seguir com os dados faltantes na base de dados e não realizar tratamentos como exclusão (podemos perder muitos dados importantes) ou substituição (dados únicos como IDs não podem ser substituídos).

Pensando nisso, vamos levar em consideração a hipótese 2, de que houve uma tentativa de esconder o dispositivo utilizado na compra. Com isso, podemos ter um dado relevante para identificar compras fraudulentas.

### 3. Análise da coluna `device_id`

Seguindo com a nossa hipótese de que os valores nulos da coluna `device_id` signficam tentativa de esconder o despositivo utilizado na compra, vamos analisar de esse campo está relacionado com a coluna `has_cbk`.

In [14]:
# Categoria para verificar se o dipositivo está presente ou não (True - presente, False - ausente)
df['device_id_presente'] = df['device_id'].notna()

# Taxa de chargeback por dispositivo presente e faltante
taxa_cbk_por_device = df.groupby('device_id_presente')['has_cbk'].mean()

print("Taxa de Chargeback (com vs. sem device_id):")
print(taxa_cbk_por_device)

Taxa de Chargeback (com vs. sem device_id):
device_id_presente
False    0.080723
True     0.136767
Name: has_cbk, dtype: float64


Com base no resultado apresentado acima, podemos perceber que a nossa hipótese inicial sobre os dados nulos na coluna `device_id` não se mostrou verdadeira, pelo contrário, tivemos uma porcentagem maior para taxa de chargeback quando o aparelho está presente (13%) do que quando está ausente (8%).



Como percebemos que o problema não é a falta de dispositivos nas transações, podemos investigar o `device_id` de uma outra perspectiva.

Nesse caso, em vez de olhar se o produto em falta tem relação com o chargeback, podemos focar na **repetição** de dispositivos nas compras.

Dessa forma, podemos identificar possíveis recorrências de chargebacks em dispositivos específicos.


Agora, vamos analisar a quantidade de transações por produtos e verificar a taxa de chargeback.

In [8]:
# Agrupar por dispositivo
device_analysis = df.groupby('device_id').agg(
    total_transacoes=('transaction_id', 'count'),
    total_cbk=('has_cbk', 'sum')
).reset_index()

# Taxa de chargeback para cada dispositivo
device_analysis['taxa_cbk'] = device_analysis['total_cbk'] / device_analysis['total_transacoes']

# Ordenar por total_transacoes
top_devices_suspeitos = device_analysis.sort_values(by='total_transacoes', ascending=False)

# Mostrar os 15 dispositivos mais usados e suas taxas de chargeback
print("Análise dos 15 dispositivos mais utilizados:")
print(top_devices_suspeitos.head(15))

Análise dos 15 dispositivos mais utilizados:
      device_id  total_transacoes  total_cbk  taxa_cbk
1133   563499.0                22         19  0.863636
693    342890.0                19         15  0.789474
198    101848.0                17         15  0.882353
884    438940.0                14         13  0.928571
1103   547440.0                13         12  0.923077
541    274282.0                 8          3  0.375000
1091   542535.0                 7          6  0.857143
1184   589318.0                 7          0  0.000000
446    223682.0                 7          4  0.571429
1909   960729.0                 6          6  1.000000
1952   978084.0                 6          4  0.666667
515    262327.0                 6          4  0.666667
1419   707086.0                 6          4  0.666667
725    357277.0                 6          6  1.000000
360    174844.0                 5          5  1.000000


Inicialmente, observamos que transações com `device_id` presente tinham uma taxa de fraude maior (13%). Com o resultado da análise acima, notamos que esse resultado foi gerado por um pequeno número de dispositivos suspeitos. Por exemplo, o `device_id` de número `563499` foi utilizado em `22` transações, das quais `86%` resultaram em chargeback, evidenciando que um único dispositivo foi utilizado em diversos casos de chargeback.

Existem até mesmo casos de dispositivos com `100%` de taxa de chargeback, como os dispositivos `960729`, `357277` e `174844`.

Além disso, também podemos verificar dispositivos com baixa taxa de chargeback, como é o caso do dispositivo `589318`, utilizado 7 vezes mas com taxa de `0%`.

Com esse resultado, podemos notar que as fraudes não ocorreram de forma espalhada aleatoriamente, mas de forma concentrada em alguns dispositivos.

Agora, podemos tomar uma ação para lidar com os dispositivos suspeitos.

Ação: qualquer `device_id` com mais de 3 transações e uma `taxa_cbk` acima de `80%` será adicionado a uma lista de bloqueio.

Pensando nisso, vamos verificar quantos dispositivos da nossa base de dados se encaixa nessa regra que criamos.

In [9]:
# Filtrar dispositivos com mais de 3 transações e taxa de chargeback acima de 80%
dispositivos_suspeitos_filtrados = top_devices_suspeitos[
    (top_devices_suspeitos['total_transacoes'] > 3) &
    (top_devices_suspeitos['taxa_cbk'] > 0.80)
]

print("Dispositivos com mais de 3 transações e taxa de chargeback acima de 80%:")
print(dispositivos_suspeitos_filtrados)

Dispositivos com mais de 3 transações e taxa de chargeback acima de 80%:
      device_id  total_transacoes  total_cbk  taxa_cbk
1133   563499.0                22         19  0.863636
198    101848.0                17         15  0.882353
884    438940.0                14         13  0.928571
1103   547440.0                13         12  0.923077
1091   542535.0                 7          6  0.857143
1909   960729.0                 6          6  1.000000
725    357277.0                 6          6  1.000000
360    174844.0                 5          5  1.000000
629    308950.0                 5          5  1.000000
1201   597308.0                 5          5  1.000000
1666   822562.0                 4          4  1.000000
238    122092.0                 4          4  1.000000
2         486.0                 4          4  1.000000
1345   670695.0                 4          4  1.000000
1147   570971.0                 4          4  1.000000
1401   698520.0                 4          4  1

Encontramos um total de 17 dispositivos. Vamos comparar com o total de dispositivos da nossa base.

In [16]:
num_unique_devices = df['device_id'].nunique()

percentual_dispositivos_suspeitos = (len(dispositivos_suspeitos_filtrados) / num_unique_devices) * 100


print(f"Os {len(dispositivos_suspeitos_filtrados)} dispositivos de alto risco representam {percentual_dispositivos_suspeitos:.2f}% do total de dispositivos únicos.")

Os 17 dispositivos de alto risco representam 0.85% do total de dispositivos únicos.


Ou seja, 17 dispositivos de um total de 1996 se encaixam na nossa regra, cerca de 0.85%.

Pode parecer um valor irrevelante comparado ao total, porém, podemos olhar mais afundo e investigar o quantos % da taxa de chargeback total esses 17 dispositivos representam.

In [17]:
# Lista de dispositivos suspeitos
lista_ids_suspeitos = dispositivos_suspeitos_filtrados['device_id']

# Impacto desses dispositivos
total_cbk_dos_17_dispositivos = dispositivos_suspeitos_filtrados['total_cbk'].sum()

# Total de chargebacks em todo o dataset
total_cbk_dataset_inteiro = df['has_cbk'].sum()

# Percentual de impacto
percentual_fraude_explicada = (total_cbk_dos_17_dispositivos / total_cbk_dataset_inteiro) * 100

In [18]:
print(f"Número de dispositivos suspeitos: {len(lista_ids_suspeitos)}")
print(f"Número total de chargebacks no dataset: {total_cbk_dataset_inteiro}")
print(f"Chargebacks causados por estes {len(lista_ids_suspeitos)} dispositivos: {total_cbk_dos_17_dispositivos}")
print(f"\nEstes {len(lista_ids_suspeitos)} dispositivos (cerca de {percentual_dispositivos_suspeitos:.2f}% do total) são responsáveis por {percentual_fraude_explicada:.2f}% de TODA a fraude no dataset!")

Número de dispositivos suspeitos: 17
Número total de chargebacks no dataset: 391
Chargebacks causados por estes 17 dispositivos: 120

Estes 17 dispositivos (cerca de 0.85% do total) são responsáveis por 30.69% de TODA a fraude no dataset!


Tivemos uma descoberta muito interessante, apenas `0.85%` dos dispositivos são responsáveis por `30.69%` dos chargebacks.

Uma regra simples de bloqueio de dispositivos baseada em reputação (mais de 3 transações e 80% de fraude) teria prevenido quase um terço das perdas por fraude, com um impacto quase nulo em clientes legítimos.

## Conclusão

A análise dos dados revelou que os casos de chargeback que não são aleatórios, mas sim concentrados.

A investigação da coluna `device_id`, a única com valores ausentes (`~25%`), nos trouxe as seguintes informações:

1.   A hipótese inicial de que a ausência de `device_id` indicaria fraude não foi confirmada. Na verdade, transações com `device_id` presente tiveram uma taxa de chargeback maior (`13.7%`) do que as ausentes (`8.1%`).

2.   Descobrimos que a maior parte dos chargebacks eram causados por um pequeno número de dispositivos. Um grupo de 17 dispositivos (menos de 1% do total de dispositivos únicos) que se enquadram em uma regra de alto risco (>3 transações e >80% de taxa de chargeback).

3.   Estes 17 dispositivos foram responsáveis, sozinhos, por 120 chargebacks, o que representa 30,69% de todo o prejuízo por fraude no dataset.


Esta análise nos mostra que a implementação de um sistema de reputação e bloqueio de dispositivos é a recomendação de maior impacto e precisão para a prevenção de fraudes nesta base de dados.

Os detalhes completos da solução e as recomendações de negócio estão no documento de apresentação principal.